참고: https://leechamin.tistory.com/114?category=836793

# Library import & version check

In [4]:
import requests
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver

import tensorflow as tf

In [11]:
# 버젼 확인
print(requests.__version__)
print(tf.__version__)

2.22.0
2.1.0


In [ ]:
# nltk에서 파일 다운로드

import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


# Scraping / Crawling

# 1. requests 패키지

## - get 방식

In [92]:
def getdownload(url, param = None, retires = 3):
    resp = None
    
    try:
        # 전달받은 결과값
        resp = requests.get(url, params = param)
        resp.raise_for_status()   # 200 OK 코드가 아닌 경우 에러
        # 예외 처리
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retires > 0:
            print('Retries : {0}'.format(retries))
            return getdownload(url, param, retries - 1)  # 재귀 호출, -1
        else:
            print(resp.status_code)  # 응답 코드
            print(resp.reason)
            print(resp.request.headers)
            print(resp.text)
            
    return resp

In [27]:
url = "https://www.crawler-test.com/status_codes/status_100"
getdownload(url)

408
REQUEST_TIMEOUT
{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


<Response [408]>

In [28]:
url = "https://www.crawler-test.com/status_codes/status_200"
getdownload(url)

<Response [200]>

## - post 방식

In [41]:
def postdownload(url, data = None, param = None, retires = 3):
    resp = None
    
    try:
        # 전달받은 결과값
        resp = requests.post(url, data, params = param)
        resp.raise_for_status()
        # 예외 처리
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retires > 0:
            print('Retries : {0}'.format(retries))
            return postdownload(url, param, retries - 1)  # 재귀 호출, -1
        else:
            print(resp.status_code)
            print(resp.reason)      
            print(resp.request.headers)
            
    return resp

In [42]:
url = "https://www.crawler-test.com/status_codes/status_200"
postdownload(url)

404
Not Found
{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '0'}


<Response [404]>

## - Cookie 활용

In [39]:
def postdownloadcookie(url, data = None, param = None, cookie = None, retires = 3):
    resp = None
    
    try:
        # 전달받은 결과값
        resp = requests.post(url, data = data, cookies = cookie, params = param)
        resp.raise_for_status()
        # 예외 처리
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retires > 0:
            print('Retries : {0}'.format(retries))
            return postdownload(url, data, param, cookie, retries - 1) 
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.headers)
            
    return resp

In [40]:
url = "https://www.crawler-test.com/status_codes/status_200"
postdownloadcookie(url)

404
Not Found
{'Content-Encoding': 'gzip', 'Content-Type': 'text/html;charset=utf-8', 'Date': 'Sat, 22 Feb 2020 04:15:36 GMT', 'Server': 'nginx/1.10.3', 'Vary': 'Accept-Encoding', 'X-Cascade': 'pass', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'Content-Length': '20', 'Connection': 'keep-alive'}


<Response [404]>

In [44]:
url = "https://pythonscraping.com/pages/files/processing.php"
data = {"firstname" : 'test', 'lastname' : 1234}

In [50]:
postdownloadcookie(url, data)

<Response [200]>

## - Session 활용

In [53]:
Session = requests.Session() # session 객체 가져옴

data = {'username': 'test', 'password' : 'password'}
html = session.post(url, data) # post 방식으로 요청
html.text

'Hello there,  !'

In [52]:
html = session.post(url)  # 재요청시 username, password 필요 없음
html.text

'Hello there,  !'

# 2. BeautifulSoup 이용한 HTML 분석

In [84]:
html = '''
<IDOCTYPE html>
<html>
    <head>
        <meta charset='utf-8'>
        <title>BeautifulSoup Training</title>
    </head>
    <body>
        <div id = 'result'>
            <p class = 'row'>
                <a class = "red">Go to page 1</a>,
                <a class = "blue">Go to page 2</a>,
                <a class = "green">Go to page 3</a>,
                <a class = "yellow">Go to page 4</a>, 
                <a class = "red">Go to page 5</a>,
                <b id = "gray">Go to pase 6</b>,
                <c id = "red">Go to pase 7</c>
            </p>
        </div>
     </body>           
</html>                
'''

In [85]:
# 객체 생성

dom = BeautifulSoup(html,'html')

In [86]:
# 서버로 html을 받으면 dom tree를 만듦

dom

<html><body><idoctype html="">
<meta charset="utf-8"/>
<title>BeautifulSoup Training</title>
<div id="result">
<p class="row">
<a class="red">Go to page 1</a>,
                <a class="blue">Go to page 2</a>,
                <a class="green">Go to page 3</a>,
                <a class="yellow">Go to page 4</a>, 
                <a class="red">Go to page 5</a>,
                <b id="gray">Go to pase 6</b>,
                <c id="red">Go to pase 7</c>
</p>
</div>
</idoctype></body></html>

# - find()
### html 태그 중심으로 검색

In [79]:
# 해당하는 태그의 첫번째

dom.find('a')

<a class="red">Go to page 1</a>

In [87]:
# 해당하는 태그 전부

dom.find_all('a')

[<a class="red">Go to page 1</a>,
 <a class="blue">Go to page 2</a>,
 <a class="green">Go to page 3</a>,
 <a class="yellow">Go to page 4</a>,
 <a class="red">Go to page 5</a>]

In [81]:
# 집합 기호 안에 attribute 검색
# attribute가 id, id가 result인 것만 검색

dom.find( '', {"id" : "result"})

<div id="result">
<p class="row">
<a class="red">Go to page 1</a>,
                <a class="blue">Go to page 2</a>,
                <a class="green">Go to page 3</a>,
                <a class="yellow">Go to page 4</a>, 
                <b id="gray">Go to pase 5</b>,
                <c id="red">Go to pase 6</c>
</p>
</div>

In [90]:
# attribute가 class, class가 red인 것만 검색
# 첫 번째만 검색

dom.find( '', {"class" : "red"})

<a class="red">Go to page 1</a>

In [89]:
# 전부 검색

dom.find_all( '', {"class" : "red"})

[<a class="red">Go to page 1</a>, <a class="red">Go to page 5</a>]

## find() 실습 1

In [99]:
# get()
# 이 상태는 dom tree가 아니기 때문에, 크롤링 불가

url = 'http://pythonscraping.com/page/page3.html'
html = getdownload(url)
html.text

404
Not Found
{'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"
  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version="XHTML+RDFa 1.0" dir="ltr"
  xmlns:content="http://purl.org/rss/1.0/modules/content/"
  xmlns:dc="http://purl.org/dc/terms/"
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:og="http://ogp.me/ns#"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:sioc="http://rdfs.org/sioc/ns#"
  xmlns:sioct="http://rdfs.org/sioc/types#"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">

<head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta charset="utf-8" />
<link rel="shortcut icon" href="http://pythonscraping.com/misc/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="viewport" content="width=device-width, initial-scale=1

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version="XHTML+RDFa 1.0" dir="ltr"\n  xmlns:content="http://purl.org/rss/1.0/modules/content/"\n  xmlns:dc="http://purl.org/dc/terms/"\n  xmlns:foaf="http://xmlns.com/foaf/0.1/"\n  xmlns:og="http://ogp.me/ns#"\n  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"\n  xmlns:sioc="http://rdfs.org/sioc/ns#"\n  xmlns:sioct="http://rdfs.org/sioc/types#"\n  xmlns:skos="http://www.w3.org/2004/02/skos/core#"\n  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">\n\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8" />\n<link rel="shortcut icon" href="http://pythonscraping.com/misc/favicon.ico" type="image/vnd.microsoft.icon" />\n<meta name="viewport" content="width=device-width, initial-scale=1" />\n<meta name="MobileOptimized" content="width" />\n<meta name="Generator" content="Drupal 7 (http://drupal.org)" />

In [100]:
# BeautifulSoup으로 dom tree 생성

dom = BeautifulSoup(html.text, 'lxml') ; dom

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN" "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html dir="ltr" version="XHTML+RDFa 1.0" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
<head profile="http://www.w3.org/1999/xhtml/vocab">
<meta charset="utf-8"/>
<link href="http://pythonscraping.com/misc/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="width" name="MobileOptimized"/>
<meta content="Drupal 7 (http://drupal.org)" name="Generator"/>
<meta content="true" name="HandheldFriendly"/>

In [105]:
# div 하위 id가 'footer'인 것만 가져오기

footer = dom.find('div', {'id' : 'footer'}) ; footer

<div id="footer">
<div class="container">
<div class="sixteen columns clearfix">
<div class="one_third">
</div>
<div class="one_third">
</div>
<div class="one_third last">
</div>
<div class="clear"></div>
<div class="clear"></div>
<div id="credits">
                2020                                 Web Scraping with Python<br/>
                                © Ryan Mitchell, All Rights Reserved. For questions about reproduction or use of any material on this site, please contact ryan.e.mitchell@gmail.com
                </div>
</div>
</div>
</div>

In [106]:
# div 상위 parent 확인 - "wrap"
# 그 id 및 속성 가져오기

parent = footer.find_parent()
parent.name, parent.attrs

('div', {'id': 'wrap'})

In [108]:
# "wrap" 하위 div 2개 출력

children = parent.find_all(recursive = False)  # 재귀 호출 금지
for row in children:
    print(row.name, row.attrs)

div {'class': ['container']}
div {'id': 'footer'}


      body
  
skip link /  wrap 

       container / footer

## find() 실습 2

In [114]:
url = 'http://pythonscraping.com/pages/page3.html'
html = getdownload(url)

In [115]:
dom = BeautifulSoup(html.text, 'lxml')

In [120]:
# tr을 전부 찾아 해당 row의 3번째 줄 전부 출력

aList = dom.find_all('tr')
for row in aList:
    print(row.find_all(recursive = False)[ 2 ].text.strip() )

Cost
$15.00
$10,000.52
$10,005.00
$0.50
$1.50


# - select()
### css 태그 중심으로 검색

In [122]:
html = '''
<IDOCTYPE html>
<html>
    <head>
        <meta charset='utf-8'>
        <title>BeautifulSoup Training</title>
    </head>
    <body>
        <div id = 'result'>
            <p class = 'row'>
                <a class = "red">Go to page 1</a>,
                <a class = "blue">Go to page 2</a>,
                <a class = "green">Go to page 3</a>,
                <a class = "yellow">Go to page 4</a>, 
                <a class = "red">Go to page 5</a>,
                <b id = "gray">Go to pase 6</b>,
                <c id = "red">Go to pase 7</c>
            </p>
        </div>
     </body>           
</html>  
'''

In [123]:
dom = BeautifulSoup(html, 'lxml') ; dom

<html><body><idoctype html="">
<meta charset="utf-8"/>
<title>BeautifulSoup Training</title>
<div id="result">
<p class="row">
<a class="red">Go to page 1</a>,
                <a class="blue">Go to page 2</a>,
                <a class="green">Go to page 3</a>,
                <a class="yellow">Go to page 4</a>, 
                <a class="red">Go to page 5</a>,
                <b id="gray">Go to pase 6</b>,
                <c id="red">Go to pase 7</c>
</p>
</div>
</idoctype></body></html>

In [124]:
# 해당하는 태그의 첫번째

dom.select_one('a')

<a class="red">Go to page 1</a>

In [125]:
# 해당하는 태그 전부

dom.select('a')

[<a class="red">Go to page 1</a>,
 <a class="blue">Go to page 2</a>,
 <a class="green">Go to page 3</a>,
 <a class="yellow">Go to page 4</a>,
 <a class="red">Go to page 5</a>]

In [127]:
# '#' : id
# '.' : class

dom.select_one('#gray')

<b id="gray">Go to pase 6</b>

## select() 실습

In [146]:
url = "https://media.daum.net/issue/5008621"
html = requests.get(url)
html_text = html.text

dom = BeautifulSoup(html.text, 'lxml')

In [157]:
dom.select('#mArticle > ul > li > div > strong.tit_thumb > a.link_txt')

[<a class="link_txt" href="http://v.media.daum.net/v/20200222152848088">청도대남병원 사망자 2명, 직접 사인은 코로나19</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222151525916">코로나19 확진 청주여성 지역 식당·마트 들러..증평 긴장 고...</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222151354890">이스라엘 성지순례단 9명도 코로나19 확진..감염경로 오리무중</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222150937820">코로나19 덮친 대구 경제 직격탄.."앞이 안 보인다"</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222150712789">강원 동부전선 육군 코로나19 비상..장병 1명 등 확진</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222145630627">은평성모병원 확진자 1명 추가..접촉자 자가격리</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222145054518">통합당 "문대통령, 어느 나라 대통령?"..'코로나19 대응'...</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222144620424">전주 방화범 코로나 증세로 일선경찰서 '긴장'..음성' 결과에...</a>,
 <a class="link_txt" href="http://v.media.daum.net/v/20200222144400386">강원 동부전선 육군 코로나19 비상..장병 1명 등 확진</a>,
 <a clas

In [166]:
# 방법 1: get_text() 이용

news_list = dom.select(
    '#mArticle > ul > li > div > strong.tit_thumb > a.link_txt')

for rows in news_list:
    print(rows.get_text())

청도대남병원 사망자 2명, 직접 사인은 코로나19
코로나19 확진 청주여성 지역 식당·마트 들러..증평 긴장 고...
이스라엘 성지순례단 9명도 코로나19 확진..감염경로 오리무중
코로나19 덮친 대구 경제 직격탄.."앞이 안 보인다"
강원 동부전선 육군 코로나19 비상..장병 1명 등 확진
은평성모병원 확진자 1명 추가..접촉자 자가격리
통합당 "문대통령, 어느 나라 대통령?"..'코로나19 대응'...
전주 방화범 코로나 증세로 일선경찰서 '긴장'..음성' 결과에...
강원 동부전선 육군 코로나19 비상..장병 1명 등 확진
당국 "격리환자 2명 위중"..다른 6명은 산소마스크 사용


In [175]:
# 방법 2: index 이용해서 text() 이용

news_list = dom.select(
    '#mArticle > ul > li > div > strong.tit_thumb > a.link_txt')

for index in range(0, len(news_list)):
    print(news_list[index].text)

청도대남병원 사망자 2명, 직접 사인은 코로나19
코로나19 확진 청주여성 지역 식당·마트 들러..증평 긴장 고...
이스라엘 성지순례단 9명도 코로나19 확진..감염경로 오리무중
코로나19 덮친 대구 경제 직격탄.."앞이 안 보인다"
강원 동부전선 육군 코로나19 비상..장병 1명 등 확진
은평성모병원 확진자 1명 추가..접촉자 자가격리
통합당 "문대통령, 어느 나라 대통령?"..'코로나19 대응'...
전주 방화범 코로나 증세로 일선경찰서 '긴장'..음성' 결과에...
강원 동부전선 육군 코로나19 비상..장병 1명 등 확진
당국 "격리환자 2명 위중"..다른 6명은 산소마스크 사용


In [176]:
# 리스트에 담기

news_lists= []

for index in range(0, len(news_list)):
    news_lists.append(news_list[index].text)
    print(news_list[index].text)

청도대남병원 사망자 2명, 직접 사인은 코로나19
코로나19 확진 청주여성 지역 식당·마트 들러..증평 긴장 고...
이스라엘 성지순례단 9명도 코로나19 확진..감염경로 오리무중
코로나19 덮친 대구 경제 직격탄.."앞이 안 보인다"
강원 동부전선 육군 코로나19 비상..장병 1명 등 확진
은평성모병원 확진자 1명 추가..접촉자 자가격리
통합당 "문대통령, 어느 나라 대통령?"..'코로나19 대응'...
전주 방화범 코로나 증세로 일선경찰서 '긴장'..음성' 결과에...
강원 동부전선 육군 코로나19 비상..장병 1명 등 확진
당국 "격리환자 2명 위중"..다른 6명은 산소마스크 사용


In [186]:
# 파일로 저장

with open('/home/ai19/news_lists.txt', 'w') as f:
    for text in news_lists:
        f.write(text + '\n')

https://code.visualstudio.com/

- deb 다운로드
- 텍스트 편집 손쉽게

# 3. Selenium 패키지

크롬 webdriver 설치

https://sites.google.com/a/chromium.org/chromedriver/downloads

- 사이트 접속
- 크롬 정보 확인
- 해당하는 버젼 다운로드

- web driver API 이용해 별도의 웹 브라우저를 호출하지 않고, 호출하는 것처럼 사용
- 내가 원하는 웹드라이버가 설치돼 있어야 함
- 어디에 풀든 상관 없지만, 사용 시 위치 기억해야 함

In [188]:
path = '/home/ai19/anaconda3/chromedriver'
driver = webdriver.Chrome(path)

In [191]:
url = 'http://example.webscraping.com/places/default/search'
driver.get(url)

In [196]:
# 시뮬레이션: korea 입력 및 클릭

driver.find_element_by_id('search_term').clear()
driver.find_element_by_id('search_term').send_keys('korea')
driver.find_element_by_id('search').click()

In [198]:
# 결과 print

results = driver.find_element_by_id('results')
for tag in results.find_elements_by_tag_name('a'):
    print(tag.text)
    print(tag.get_attribute('href'))

North Korea
http://example.webscraping.com/places/default/view/North-Korea-165
South Korea
http://example.webscraping.com/places/default/view/South-Korea-211


In [202]:
# 구글에 검색

driver.get('http://ww.google.co.kr')
search = driver.find_element_by_name('q')
search.send_keys('코로나')
search.submit()

In [205]:
user = "facebook id"
pwd = "pwd"

driver.get('http://www.facebook.com')

element = driver.find_element_by_id('email')
element.send_keys(user)
element = driver.find_element_by_id('pass')
element.send_keys(pwd)
element.send_keys(keys.RETURN)